# Documentation 
https://github.com/innovata/DSE-REST-API  

# [1] 3DEXPERIENCE Platform CLM Agent 생성  
- "F:\pypjts\DSE-REST-API\How to use APIs\credentials"경로에 Agent ID, Password 저장 

# [2] 파이썬 가상환경(Python Virtual Environment) 생성 및 dserestapi 패키지 설치



In [120]:
# 개발용 경로설정
import os, sys 
sys.path.append(r"F:\pypjts\DSE-REST-API\src")

In [ ]:
# 커맨드 프롬프트 사용 권장
# !pip install dserestapi

# [3] 환경셋업

In [118]:
import os
import re 
os.environ["CLM_AGENT_CREDENTIAL_PATH"] = r"F:\pypjts\DSE-REST-API\How to use APIs\credentials\CML Agent ID-PW.json"
os.environ["3DX_PLATFORM_TENANT_URI"] = "https://r1132100527066-apk2-sgi.3dexperience.3ds.com:443"
for k, v in os.environ.items():
    if re.search(r"^CLM|^3DX", k):
        print([k, v])

['CLM_AGENT_CREDENTIAL_PATH', 'F:\\pypjts\\DSE-REST-API\\How to use APIs\\credentials\\CML Agent ID-PW.json']
['3DX_PLATFORM_TENANT_URI', 'https://r1132100527066-apk2-sgi.3dexperience.3ds.com:443']


In [119]:
import dserestapi 
dserestapi

<module 'dserestapi' from 'F:\\pypjts\\DSE-REST-API\\src\\dserestapi\\__init__.py'>

# [4] 사용 예제 | Storages

In [123]:
from dserestapi import Storages
api = Storages()
api 

### [4.1] 스토리지 목록 가져오기 

In [124]:
# 모든 스토리지 리스트 가져오기
res = api.get()
res

<Response [200]>

In [125]:
import json 
file = os.path.join(os.getcwd(), "01_Storages", "Storages List.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

In [126]:
# !pip install pandas 

In [142]:
import pandas as pd 
df = pd.DataFrame(res.json()['cards'])
df#.head(5)

,id,resourceId,projectId,resourceUUID,workspaceId,name,creator,kind,type,created,lastModified,permissions
0,0add512f-3759-487b-aa40-9c5019874ea3,20241217_16:00_dsjeon_storage,dp-global-000000,0add512f-3759-487b-aa40-9c5019874ea3,dw-global-000000-default,20241217_16:00_dsjeon_storage,jeondongsoo,Storage,ObjectStorage,1734418711011,1734418711007,"{'read': True, 'write': True, 'execute': True}"
1,7118465e-739b-4746-8fc7-657755764ee7,240129_part_information_OU3,dp-global-000000,7118465e-739b-4746-8fc7-657755764ee7,dw-global-000000-default,240129_part_information_OU3,N/A,Storage,IndexUnit,1707553208946,1745657169549,"{'read': True, 'write': True, 'execute': True}"
2,R1132100527066_3DS_AdditiveManufacturingQualit...,3DS_AdditiveManufacturingQualityAnalyst_Data,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,R1132100527066_3DS_AdditiveManufacturingQualit...,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,3DS_AdditiveManufacturingQualityAnalyst_Data,svc_sgi_clouduser,Storage,ObjectStorage,1751697876373,1751697876334,"{'read': True, 'write': True, 'execute': True}"
3,R1132100527066_3DS_AdditiveManufacturingQualit...,3DS_AdditiveManufacturingQualityAnalyst_Image,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,R1132100527066_3DS_AdditiveManufacturingQualit...,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,3DS_AdditiveManufacturingQualityAnalyst_Image,svc_sgi_clouduser,Storage,ObjectStorage,1751697876576,1751697876537,"{'read': True, 'write': True, 'execute': True}"
4,R1132100527066_3DS_AdditiveManufacturingQualit...,3DS_AdditiveManufacturingQualityAnalyst_Result,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,R1132100527066_3DS_AdditiveManufacturingQualit...,ca221e4a-46b1-43b1-9c3e-4eea8a0aa662,3DS_AdditiveManufacturingQualityAnalyst_Result,svc_sgi_clouduser,Storage,ObjectStorage,1751697876473,1751697876435,"{'read': True, 'write': True, 'execute': True}"
...,...,...,...,...,...,...,...,...,...,...,...,...
146,5841fd29-a3a3-4780-8343-b1fe0ccaae71,r2vsurvey,dp-r2vsurvey-000000,5841fd29-a3a3-4780-8343-b1fe0ccaae71,dw-r2vsurvey-000000-default,r2vsurvey,svc_sgi_clouduser,Storage,IndexUnit,1745666684450,1751707573630,"{'read': True, 'write': False, 'execute': False}"
147,3fbd978d-c9ce-43af-9f73-ea4aed3ab4b5,sgi,dp-global-000000,3fbd978d-c9ce-43af-9f73-ea4aed3ab4b5,dw-global-000000-default,sgi,jihunyoon,Storage,IndexUnit,1734678234116,1734678234102,"{'read': True, 'write': True, 'execute': True}"
148,b35600d0-6c8d-4142-9e48-d35ab335d62b,sgi_test_syyou,dp-global-000000,b35600d0-6c8d-4142-9e48-d35ab335d62b,dw-global-000000-default,sgi_test_syyou,mobilevm,Storage,IndexUnit,1734483191336,1734483191331,"{'read': True, 'write': True, 'execute': True}"
149,595464cb-65bd-44d9-b719-c08b1a605070,syyou_storage,dp-global-000000,595464cb-65bd-44d9-b719-c08b1a605070,dw-global-000000-default,syyou_storage,mobilevm,Storage,ObjectStorage,1734677241229,1734677241224,"{'read': True, 'write': True, 'execute': True}"


In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            151 non-null    object
 1   resourceId    151 non-null    object
 2   projectId     151 non-null    object
 3   resourceUUID  151 non-null    object
 4   workspaceId   151 non-null    object
 5   name          151 non-null    object
 6   creator       151 non-null    object
 7   kind          151 non-null    object
 8   type          151 non-null    object
 9   created       151 non-null    int64 
 10  lastModified  151 non-null    int64 
 11  permissions   151 non-null    object
dtypes: int64(2), object(10)
memory usage: 14.3+ KB


In [139]:
df.dtypes

id              object
resourceId      object
projectId       object
resourceUUID    object
workspaceId     object
name            object
creator         object
kind            object
type            object
created          int64
lastModified     int64
permissions     object
dtype: object

In [132]:
df.describe()

,created,lastModified
count,1.510000e+02,1.510000e+02
mean,1.734608e+12,1.741621e+12
std,1.561000e+10,1.020891e+10
min,1.707553e+12,1.707561e+12
25%,1.730700e+12,1.734400e+12
50%,1.734419e+12,1.745657e+12
75%,1.751698e+12,1.751698e+12
max,1.753677e+12,1.753692e+12


In [141]:
workspace_ids = df.workspaceId.unique()
import pprint 
pp = pprint.PrettyPrinter(indent=2)
print("Unique Workspace IDs Count:", len(workspace_ids))
pp.pprint(sorted(workspace_ids))

Unique Workspace IDs Count: 29
[ '1d99889d-91a9-47c6-ae10-5a0f0c06bee5',
  '20f147c2-5b24-4a71-af10-81521fbcf73a',
  '6cc6c82c-1181-4b51-9af5-75865de628b2',
  '6cdce84b-03c4-4ca9-a158-ab58df7ac517',
  'a818380d-3daf-40b4-91f0-4667fcfd0c8c',
  'bf86652d-9049-4083-8e5e-c0e0a25d8350',
  'ca221e4a-46b1-43b1-9c3e-4eea8a0aa662',
  'dp-bioviainsightqueries-000000',
  'dp-bioviamaterialsintelligence-000000',
  'dp-fieldqualityintelligence-000000',
  'dp-sustainabilityexternaldata-000000',
  'dw-3dnetwork-000000-default',
  'dw-3dorganization-000000-default',
  'dw-3dplan-000000-default',
  'dw-3dspace-000000-default',
  'dw-6645-4ddb-stocktraderstudio',
  'dw-6645-4ddb-youtubedataanalytics',
  'dw-6wtags-000000-default',
  'dw-bioexperiment-000000-default',
  'dw-dseagent-716118-workspace',
  'dw-global-000000-default',
  'dw-inventory-000000-default',
  'dw-jle69ptoyproject-874623-youtubedataanalytics',
  'dw-materials-000000-default',
  'dw-molecularsimulation-000000-default',
  'dw-netvibes

### [4.2] ObjectStorage 신규 생성 

In [40]:
# 스토리지 생성
res = api.create(
    stype="ObjectStorage",
    name="TestObject-01.CreatedByRestAPI",
    description="테스트 후 바로 삭제할 것"
)
res 


응답코드--> 201


<Response [201]>

In [41]:
file = os.path.join(os.getcwd(), "01_Storages", "Create A Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [4.3] 생성한 스토리지 찾기

In [42]:
# 스토리지 검색-1
res = api.search_by_name(
    name="TestObject-01.CreatedByRestAPI", 
    # workspace_id="dw-global-000000-default"
)
res 


응답코드--> 200


<Response [200]>

In [43]:
file = os.path.join(os.getcwd(), "01_Storages", "Search-1 the Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [4.4] 스토리지 비우기 
- 샘플파일 아무거나 수동으로 업로드

In [44]:
resource_uuid = "4407fe2c-1739-4391-9843-d120d7fe6cd7"

In [45]:
res = api.clear(resource_uuid)
res 


응답코드--> 204


<Response [204]>

- res.json() 없음 --> 결과는 플랫폼에서 직접 확인  

### [4.5] 스토리지 Export

In [47]:
res = api.export_(resource_uuid)
res 


응답코드--> 200


<Response [200]>

In [48]:
file = os.path.join(os.getcwd(), "01_Storages", "Export the Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [4.6] 스토리지 Import

In [36]:
payload = {
    "name": "TestObject-05.ImportByRestAPI",
    "identifier": "TestObject-05.ImportByRestAPI",
    "update": True,
    "content": "ewogICJzdG9yYWdlIiA6IHsKICAgICJAY2xhc3MiIDogIk9iamVjdFN0b3JhZ2UiLAogICAgInJlc291cmNlSWQiIDogIlRlc3RPYmplY3QtMDMuQ3JlYXRlZEJ5UmVzdEFQSSIsCiAgICAibmFtZSIgOiAiVGVzdE9iamVjdC0wMy5DcmVhdGVkQnlSZXN0QVBJIiwKICAgICJkZXNjcmlwdGlvbiIgOiAi7YWM7Iqk7Yq4IO2bhCDrsJTroZwg7IKt7KCc7ZWgIOqygyIsCiAgICAia2luZCIgOiAiU3RvcmFnZSIsCiAgICAidHlwZSIgOiAiT2JqZWN0U3RvcmFnZSIsCiAgICAicmVzb3VyY2VQYXRoIiA6ICJTdG9yYWdlL1Rlc3RPYmplY3QtMDMuQ3JlYXRlZEJ5UmVzdEFQSS5qc29uIiwKICAgICJjb25maWciIDogewogICAgICAiYnVja2V0IiA6ICJ0ZXN0b2JqZWN0MDNjcmVhdGVkYnlyZXN0YXBpIiwKICAgICAgImRlbGVnYXRpb25zIiA6IFsgewogICAgICAgICJuYW1lIiA6ICJTZW1hbnRpY0dyYXBoSW5kZXguRE9TQWNjb3VudC4yIiwKICAgICAgICAicGVybWlzc2lvbiIgOiAicmVhZCIKICAgICAgfSwgewogICAgICAgICJuYW1lIiA6ICJTZW1hbnRpY0dyYXBoSW5kZXguRE9TQWNjb3VudC4yIiwKICAgICAgICAicGVybWlzc2lvbiIgOiAid3JpdGUiCiAgICAgIH0sIHsKICAgICAgICAibmFtZSIgOiAiUFBFUy5Eb3NBY2NvdW50IiwKICAgICAgICAicGVybWlzc2lvbiIgOiAiZXhlY3V0ZSIKICAgICAgfSwgewogICAgICAgICJuYW1lIiA6ICJEYXRhc2V0R292ZXJuYW5jZSIsCiAgICAgICAgInBlcm1pc3Npb24iIDogInJlYWQiCiAgICAgIH0gXQogICAgfQogIH0sCiAgInZlcnNpb24iIDogewogICAgInZhbHVlIiA6ICIxLjIiCiAgfQp9"
}

In [35]:
from importlib import reload
import dserestapi
reload(dserestapi)
from dserestapi import Storages
api = Storages()
res = api.import_(payload)
res 


응답코드--> 400


응답코드가 2xx 또는 3xx 대역이 아닐 경우 아래와 같이 표시됩니다-->
{ '_content': b'{"type":"about:blank","title":"Bad Request","status":400,"de'
              b'tail":"Illegal argument error Validation failed on \'ObjectSt'
              b"orage.name' with message: 'A storage with this name already "
              b'exists\'","instance":"/data-factory/api/1/storage/import"}',
  '_content_consumed': True,
  '_next': None,
  'connection': <requests.adapters.HTTPAdapter object at 0x000001DE5AD33D70>,
  'cookies': <RequestsCookieJar[Cookie(version=0, name='JSESSIONID', value='FCC5CAEB6D3F33A335FE70F1A38516A9', port=None, port_specified=False, domain='r1132100527066-apk2-sgi.3dexperience.3ds.com', domain_specified=False, domain_initial_dot=False, path='/data-factory', path_specified=True, secure=True, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None, 'SameSite': 'None'}, rfc2109=False), Cookie(version=0, name='SERVERID', value='IngestionVM_0_7026', port=None, port_s

<Response [400]>

### [4.7] 생성한 스토리지 삭제

In [ ]:
# API를 이용해서 재검색
res = api.search_by_name(
    name="TestObject-01.CreatedByRestAPI"
)
resource_uuid = res.json()["cards"][0]["resourceUUID"]
resource_uuid

'4407fe2c-1739-4391-9843-d120d7fe6cd7'

In [54]:
res = api.delete(resource_uuid=resource_uuid)
res 


응답코드--> 200


<Response [200]>

- res.json() --> 삭제 함수 응답에는 JSON 데이터가 없기 때문에 에러 발생

# [5] 사용 예제 | Object Storage

### [5.1] Object Storage에 파일 업로드 

In [ ]:
# 신규 스토리지 재생성 
from dserestapi import Storages
api = Storages()
res = api.create(
    stype="ObjectStorage",
    name="TestObject-01.CreatedByRestAPI",
    description="테스트 후 바로 삭제할 것"
)
res.json()

In [82]:
# 생성한 스토리지의 resourceUUID 추출 
resource_uuid = res.json()["resourceUUID"]
# resource_uuid = "4407fe2c-1739-4391-9843-d120d7fe6cd7"
resource_uuid

'2ab30c85-f1c6-4696-9bcc-cacef848ec19'

#### [5.1.1] 파일 업로드 | 경로 미지정
- 업로드할 파일은 항상 파일 절대경로를 사용해야함

In [ ]:
from dserestapi import ObjectStorage
api = ObjectStorage()
uploading_file = r"F:\pypjts\DSE-REST-API\How to use APIs\credentials\CML Agent ID-PW.json"
res = api.upload(resourceUUID=resource_uuid, file=uploading_file, path=None)
res 

업로드할 파일: CML Agent ID-PW.json

DFS 스토리지상의 파일 절대경로: 'F:/pypjts/DSE-REST-API/How to use APIs/credentials/CML Agent ID-PW.json'

응답코드--> 200


<Response [200]>

#### [5.1.2] 파일 업로드 | 경로 강제 지정

In [ ]:
api = ObjectStorage()
uploading_file = r"F:\pypjts\DSE-REST-API\How to use APIs\credentials\CML Agent ID-PW.json"
res = api.upload(resourceUUID=resource_uuid, file=uploading_file, path="Manual_Path_01/")
res 

업로드할 파일: CML Agent ID-PW.json

DFS 스토리지상의 파일 절대경로: 'Manual_Path_01/CML Agent ID-PW.json'

응답코드--> 200


<Response [200]>

- JSON 데이터 없음

### [5.2] Object Storage에 디렉토리 내의 모든 파일들을 한번에 업로드 

In [ ]:
# 업로드할 디렉토리 확인
import os 
top_dir = r"E:\__ENGINEER_DRIVE__\__DATASET__\__Dummy Data__"
uploading_files = []
for root, dirs, files in os.walk(top_dir):
    for file in files:
        uploading_files.append(os.path.join(root, file))

uploading_files

['E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\trade_money_mv20.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\all_dtypes.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\json_list.json',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\_Boolean.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\_Datetime.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\_Datetime.json',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\_Number.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Basic Data Types\\_String.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Data Ingestion\\Data Engineer.ipynb',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Data Ingestion\\DummyData.json',
 'E:\\__ENGINEER_DRIVE__\\__DATASET__\\__Dummy Data__\\Data Ingestion\\sample01.csv',
 'E:\\__ENGINEER_DRIVE__\\__DATASE

#### [5.2.1] 멀티 파일 업로드 | 자동 경로 설정

In [ ]:
# 멀티 파일 업로드 | 자동 경로 설정
import pprint 
pp = pprint.PrettyPrinter(indent=2)
api = ObjectStorage()
res_li = api.upload_files(resourceUUID=resource_uuid, files=uploading_files, path=None)
pp.pprint(res_li)

Uploading:  31%|███       | 15/49 [00:00<00:00, 114.05it/s]


응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200


Uploading:  55%|█████▌    | 27/49 [00:00<00:00, 108.70it/s]


응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200


Uploading:  78%|███████▊  | 38/49 [00:00<00:00, 51.00it/s] 


응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200


Uploading:  94%|█████████▍| 46/49 [00:01<00:00, 24.52it/s]


응답코드--> 200

응답코드--> 200


Uploading: 100%|██████████| 49/49 [00:01<00:00, 24.70it/s]


응답코드--> 200

응답코드--> 200
[ <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [2

#### [5.2.2] 멀티 파일 업로드 | 수동 경로 설정

In [ ]:
# 멀티 파일 업로드 | 수동 경로 설정
api = ObjectStorage()
res_li = api.upload_files(resourceUUID=resource_uuid, files=uploading_files, path="Multi/")
pp.pprint(res_li)

업로드할 파일: trade_money_mv20.csv

DFS 스토리지상의 파일 절대경로: 'Multi/trade_money_mv20.csv'
업로드할 파일: all_dtypes.csv

DFS 스토리지상의 파일 절대경로: 'Multi/all_dtypes.csv'
업로드할 파일: json_list.json

DFS 스토리지상의 파일 절대경로: 'Multi/json_list.json'
업로드할 파일: _Boolean.csv

DFS 스토리지상의 파일 절대경로: 'Multi/_Boolean.csv'
업로드할 파일: _Datetime.csv

DFS 스토리지상의 파일 절대경로: 'Multi/_Datetime.csv'
업로드할 파일: _Datetime.json

DFS 스토리지상의 파일 절대경로: 'Multi/_Datetime.json'
업로드할 파일: _Number.csv

DFS 스토리지상의 파일 절대경로: 'Multi/_Number.csv'
업로드할 파일: _String.csv

DFS 스토리지상의 파일 절대경로: 'Multi/_String.csv'
업로드할 파일: Data Engineer.ipynb

DFS 스토리지상의 파일 절대경로: 'Multi/Data Engineer.ipynb'
업로드할 파일: DummyData.json

DFS 스토리지상의 파일 절대경로: 'Multi/DummyData.json'
업로드할 파일: sample01.csv

DFS 스토리지상의 파일 절대경로: 'Multi/sample01.csv'
업로드할 파일: sample01.json

DFS 스토리지상의 파일 절대경로: 'Multi/sample01.json'
업로드할 파일: sample01_even.json

DFS 스토리지상의 파일 절대경로: 'Multi/sample01_even.json'
업로드할 파일: Data Engineer.ipynb

DFS 스토리지상의 파일 절대경로: 'Multi/Data Engineer.ipynb'
업로드할 파일: Data_Modeling.json

DFS

Uploading:   0%|          | 0/49 [00:00<?, ?it/s]


응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200


Uploading:  61%|██████    | 30/49 [00:00<00:00, 222.50it/s]


응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200

응답코드--> 200


Uploading: 100%|██████████| 49/49 [00:01<00:00, 34.04it/s] 


응답코드--> 200
[ <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>,
  <Response [200]>]


- res.json() 없음 --> 업로드 결과는 플랫폼에서 직접 확인  

### [5.3] 스토리지 EXPORT 

In [94]:
api = Storages()
res = api.export_(resource_uuid=resource_uuid)
res 


응답코드--> 200


<Response [200]>

In [95]:
file = os.path.join(os.getcwd(), "01_Storages", "Export Object Storage.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

# [6] 사용 예제 | Semantic Graph Index Unit

### [6.1] SGI Storage 신규 생성시 데이터 모델링을 동시에 수행 

In [96]:
# 데이터 모델링 
data_modeling_config = {
    "datamodel": {
        "classes": [
            {
                "name": "DummyData_01",
                "parents": [],
                "pkg": "Rawdata",
                "attributes": [
                    {
                        "name": "seq",
                        "type": {
                            "dataType": "Integer",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "title",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "_text",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    }
                ]
            },
            {
                "name": "DummyData_02",
                "parents": [],
                "pkg": "Rawdata",
                "attributes": [
                    {
                        "name": "seq",
                        "type": {
                            "dataType": "Integer",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "title",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    },
                    {
                        "name": "_text",
                        "type": {
                            "dataType": "String",
                            "dataStructure": "Singleton"
                        },
                        "annotation": {}
                    }
                ]
            }
        ]
    }
}

In [ ]:
from dserestapi import Storages
# 스토리지 생성
api = Storages()
res = api.create(
    stype="IndexUnit",
    name="TestSGI_01_CreatedByRestAPI",
    description="SGI 데이터 모델링 테스트 후 삭제할 것",
    config=data_modeling_config
)
res.json() 

{'@class': 'IndexUnit',
 'resourceId': 'TestSGI_01_CreatedByRestAPI',
 'name': 'TestSGI_01_CreatedByRestAPI',
 'description': 'SGI 데이터 모델링 테스트 후 삭제할 것',
 'kind': 'Storage',
 'type': 'IndexUnit',
 'resourcePath': 'Storage/TestSGI_01_CreatedByRestAPI.json',
 'resourceUUID': 'd643b7a8-a722-49a4-a8bc-b7bc7eaa7e39',
 'projectId': 'dp-global-000000',
 'workspaceId': 'dw-global-000000-default',
 'tenant': 'R1132100527066',
 'created': '2025-07-17T10:55:01.924863Z',
 'modified': '2025-07-28T08:40:24.874939Z',
 'creator': 'jle69_gmail',
 'ceStamp': '3649ffe5-18ab-4196-ae0a-b6f89e7199d8',
 'id': 'd643b7a8-a722-49a4-a8bc-b7bc7eaa7e39',
 'config': {'datamodel': {'classes': [{'name': 'DummyData_01',
     'parents': [],
     'pkg': 'Rawdata',
     'attributes': [{'name': 'seq',
       'type': {'dataType': 'Integer', 'dataStructure': 'Singleton'},
       'annotation': {}},
      {'name': 'title',
       'type': {'dataType': 'String', 'dataStructure': 'Singleton'},
       'annotation': {}},
      {'na

In [98]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Create A SGI.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [6.2] 데이터 모델 스키마에 맞는 데이터 생성 후 스키마에 맞는 데이터 타입인지 정합성 체크 

In [101]:
resource_uuid = res.json()["resourceUUID"]
# resource_uuid = "b1176f72-a3e2-4afc-9b02-543859e0cdf2"
resource_uuid

'd643b7a8-a722-49a4-a8bc-b7bc7eaa7e39'

In [ ]:
# from dserestapi import SemanticGraphIndex
# sgi = SemanticGraphIndex()
# res = sgi.class_count(resourceUUID=resource_uuid, pkg_name="Rawdata", class_name_li=["DummyData_01"])
# with open(os.path.join(os.getcwd(), "Class Count.json"), "w", encoding="utf-8") as f:
#     json.dump(res.json(), f, ensure_ascii=False, indent=2)

In [108]:
# 데이터 생성 
data = [
    {
        "uri": "Rawdata.DummyData_01.0001",
        "class": "Rawdata.DummyData_01",
        "seq": 1,
        "title": "This is a title-1",
        "_text": "bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla~"
    },
    {
        "uri": "Rawdata.DummyData_01.0002",
        "class": "Rawdata.DummyData_01",
        "seq": 2,
        "title": "This is a title-2",
        "_text": "(2) bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla bla~"
    },
]

In [109]:
# 저장할 데이터의 정합성 체크 
from dserestapi import SemanticGraphIndex
sgi = SemanticGraphIndex()
res = sgi.validateItemsEvent(resourceUUID=resource_uuid, action="AddOrReplaceItem", data=data)
res.json()


응답코드--> 200


[]

In [105]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Validation Items.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

### [6.3] 생성한 SGI의 타겟 데이터 모델에 데이터를 저장

In [ ]:
res = sgi.ingest(resourceUUID=resource_uuid, data=data)
res.json()

{'notifications': []}

In [ ]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "Ingest Data.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)

- 데이터가 저장되었는지 플랫폼에서 직접확인

### [6.4] 데이터 저장 결과 알림 받기 

In [ ]:
res = sgi.notification(resourceUUID=resource_uuid)
res.json()

{'notifications': [], 'count': 0}

### [6.5] 저장한 데이터의 object 별 uri 가져오기

In [116]:
res = sgi.get_uri(resourceUUID=resource_uuid)
res.json()


응답코드--> 200


{'uris': ['Rawdata.DummyData_01.0001', 'Rawdata.DummyData_01.0002']}

In [117]:
file = os.path.join(os.getcwd(), "03_SemanticGraphIndex", "SGI URIs.json")
with open(file, "w", encoding="utf-8") as f:
    json.dump(res.json(), f, ensure_ascii=False, indent=2)